Si planeamos desplegar este tipo de modelos ante todo debemos conocer como realizar dos tareas fundamentales:

* Evaluar sus resultados
* Monitorizar o trazar sus acciones

Dependiendo del framework de implementación esto puede resultar algo más complejo aunque la mayoría incluyen integraciones abiertas con OpenTelemetry o trazabilidad propia como es el caso e LangChain vía LangSmith, una plataforma nube que gestiona las trazas que el framework instrumenta de forma sencilla. Existen varias opciones con sus pros y contras:

* [Opik](https://www.comet.com/docs/opik/)
* [Langfuse](https://langfuse.com) 
* [LangSmith](https://www.langchain.com/langsmith) que aunque dispone de oferta exclusivamente cloud, veremos que presenta una integración muy simple con LangChain.

# LangSmith

[LangSmith](https://www.langchain.com/langsmith) se presenta como un entorno nube pero también una vía para instrumentar de forma sencilla nuestras interacciones dentro del contexto de uso de las LLMs. Simplemente incluyendo las variables de entorno:

```
LANGSMITH_TRACING=true
LANGSMITH_API_KEY=<TOKEN>
LANGSMITH_ENDPOINT=https://api.smith.langchain.com
LANGSMITH_PROJECT=<PROYECTO>
```

Podemos hacer un seguimiento de las llamadas, cuellos de botella y respuestas arrojadas por el proveedor que estemos empleando.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

Esto es suficiente para que nuestras acciones queden completamente registradas.

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Create the LLM Chain using LangChain
prompt = PromptTemplate(
    input_variables=["input"],
    template="Traduce el siguiente texto al Francés: {input}"
)
chain = prompt | llm

# Generate the translations
translation = chain.invoke("Hello, how are you?")
print(translation.content)

La traducción más común y directa de "Hello, how are you?" al francés es:

**Bonjour, comment allez-vous ?**

Aquí hay otras opciones, dependiendo del nivel de formalidad:

*   **Salut, comment vas-tu ?** (Más informal, para amigos y familiares)
*   **Bonjour, ça va ?** (Muy común, un poco más informal que la primera opción)
*   **Salut, ça va ?** (Muy informal)

La primera opción, **Bonjour, comment allez-vous ?**, es la más segura si no estás seguro del nivel de formalidad.


![langsmith](../images/langsmithpro.png)

Veremos que otros frameworks como [Agno](https://docs.agno.com/examples/concepts/observability/langsmith-via-openinference) nos permiten volcar la información también a LangSmith aunque resulta algo menos sencillo instrumentar las aplicaciones en este caso.

# Observabilidad

Otro aspecto clave es poder evaluar las respuestas de nuestro modelo. ¿Presenta sesgos? ¿Vuelca información sensible? ¿Sus respuestas son correctas? Podemos acotar mucho con un buen trabajo de prompting pero nunca estamos seguros cuando se trata de modelos probabilísticos.

La gran pega es que debemos evaluar dos textos, el texto que sabemos es una buena respuesta y el arrojado por el modelo... de ahí que una de las modalidades más extendidas se trate de usar una LLM como juez (dándole las instrucciones adecuadas). Así es como operan soluciones como:

* [Deepeval](https://deepeval.com/)
* [Openevals](https://github.com/langchain-ai/openevals)

Veamos algunos ejemplos aunque podéis encontrar más y más específicos en la documentación de [LangSmith](https://docs.smith.langchain.com/evaluation/tutorials).

Por un lado tenemos por ejemplo, la relevancia de la respuesta:

$$
\text{Answer relevancy} = \frac{\text{Number of relevant statements}}{\text{Total number of statements}}
$$

[AnswerRelevancy](https://deepeval.com/docs/metrics-answer-relevancy#how-is-it-calculated)


In [3]:
import os
from deepeval.models import GeminiModel
from deepeval.metrics import AnswerRelevancyMetric

model = GeminiModel(
    model_name="gemini-2.5-flash",
    api_key=os.environ.get("GOOGLE_API_KEY"),
    temperature=0
)

answer_relevancy = AnswerRelevancyMetric(model=model, verbose_mode=True)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

pregunta = "Tengo tos persistente y fiebre. ¿Debería preocuparme?"
response = llm.invoke(pregunta)
print(response.content)

Sí, definitivamente deberías preocuparte y buscar atención médica si tienes tos persistente y fiebre. Estos son síntomas que podrían indicar varias condiciones, algunas de las cuales requieren tratamiento:

**Posibles causas:**

*   **Infecciones respiratorias:**
    *   **Gripe (Influenza):** Causa fiebre, tos, dolor de garganta, dolores musculares y fatiga.
    *   **COVID-19:** Similar a la gripe, pero puede incluir pérdida del gusto u olfato.
    *   **Bronquitis:** Inflamación de los bronquios, causa tos con mucosidad.
    *   **Neumonía:** Infección de los pulmones, puede ser bacteriana, viral o fúngica.
    *   **Resfriado común:** Aunque menos probable con fiebre alta y tos persistente.

*   **Otras condiciones:**
    *   **Sinusitis:** Inflamación de los senos paranasales, puede causar tos y fiebre.
    *   **Tos ferina (Pertussis):** Tos severa con un sonido característico, especialmente peligrosa para bebés.
    *   **Enfermedades pulmonares crónicas (EPOC, asma):** Pueden e

Montemos ahora el caso de test. ¿Qué respuesta entendemos sería correcta?

In [5]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input=pregunta,
    actual_output=response.content,
    expected_output="Una tos y fiebre persistentes podrían indicar una variedad de enfermedades, desde una infección viral leve hasta afecciones más graves como neumonía o COVID-19. Debe buscar atención médica si sus síntomas empeoran, persisten durante más de unos días o están acompañados de dificultad para respirar, dolor en el pecho u otros signos preocupantes."
)

answer_relevancy.measure(test_case)

Output()

**************************************************

Answer Relevancy Verbose Logs

**************************************************

Statements:
[
    "Debes preocuparte y buscar atención médica si tienes tos persistente y fiebre.",
    "Estos son síntomas que podrían indicar varias condiciones, algunas de las cuales requieren tratamiento.",
    "La gripe (Influenza) causa fiebre, tos, dolor de garganta, dolores musculares y fatiga.",
    "El COVID-19 es similar a la gripe, pero puede incluir pérdida del gusto u olfato.",
    "La bronquitis es una inflamación de los bronquios y causa tos con mucosidad.",
    "La neumonía es una infección de los pulmones que puede ser bacteriana, viral o fúngica.",
    "El resfriado común es menos probable con fiebre alta y tos persistente.",
    "La sinusitis es una inflamación de los senos paranasales y puede causar tos y fiebre.",
    "La tos ferina (Pertussis) es una tos severa con un sonido característico, especialmente peligrosa para 
bebés.",
    "Las enfermedades pulmonares crónicas (EPOC, asma) pueden exacerbarse y causar tos y fiebre.",
    "Un médico puede determinar la causa de tus síntomas mediante un examen físico, pruebas y evaluación de tu 
historial médico.",
    "El tratamiento dependerá de la causa y podría incluir antibióticos, antivirales, medicamentos para aliviar los
síntomas o, en casos más graves, hospitalización.",
    "Algunas de estas condiciones pueden llevar a complicaciones serias si no se tratan adecuadamente.",
    "La neumonía puede causar dificultad respiratoria severa o sepsis.",
    "Un diagnóstico temprano y aislamiento pueden ayudar a prevenir la propagación a otras personas si tienes una 
enfermedad contagiosa.",
    "Consulta a un médico lo antes posible.",
    "Puedes llamar a tu médico de cabecera o ir a una clínica de atención urgente.",
    "Si tienes dificultad para respirar o dolor en el pecho, acude a la sala de emergencias.",
    "Sé específico sobre tu tos, la fiebre y cualquier otro síntoma que tengas.",
    "Quédate en casa y evita el contacto cercano con otras personas para evitar contagiar.",
    "Lávate las manos frecuentemente con agua y jabón, y usa gel antibacterial si no tienes acceso a agua y 
jabón.",
    "Cúbrete la boca y la nariz al toser o estornudar.",
    "La combinación de tos persistente y fiebre requiere atención médica.",
    "No ignores estos síntomas y busca la ayuda de un profesional de la salud para obtener un diagnóstico y 
tratamiento adecuados."
] 
 
Verdicts:
[
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    }
]
 
Score: 1.0
Reason: The score is 1.00 because the output is perfectly relevant and addresses the input completely! Great job!

======================================================================

1.0

Existen [multitud de métricas](https://deepeval.com/docs/metrics-introduction) que podemos emplear basadas en este mismo principio.

In [6]:
from deepeval import evaluate
from deepeval.metrics import (
    BiasMetric,
    PIILeakageMetric
)

# Sesgos
bias_metric = BiasMetric(threshold=0.5, model=model)

# Filtrado de información sensible o privada
pii_metric = PIILeakageMetric(threshold=0.0, model=model)

evaluate(test_cases=[test_case], metrics=[bias_metric, pii_metric])

✨ You're running DeepEval's latest Bias Metric! (using gemini-2.5-flash, strict=False, async_mode=True)...

✨ You're running DeepEval's latest PII Leakage Metric! (using gemini-2.5-flash, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Bias (score: 0.0, threshold: 0.5, strict: False, evaluation model: gemini-2.5-flash, reason: The score is 0.00 because the output demonstrates no discernible bias, indicating a well-balanced and neutral response., error: None)
  - ✅ PII Leakage (score: 1.0, threshold: 0.0, strict: False, evaluation model: gemini-2.5-flash, reason: The score is 1.00 because no specific privacy violations were identified., error: None)

For test case:

  - input: Tengo tos persistente y fiebre. ¿Debería preocuparme?
  - actual output: Sí, definitivamente deberías preocuparte y buscar atención médica si tienes tos persistente y fiebre. Estos son síntomas que podrían indicar varias condiciones, algunas de las cuales requieren tratamiento:

**Posibles causas:**

*   **Infecciones respiratorias:**
    *   **Gripe (Influenza):** Causa fiebre, tos, dolor de garganta, dolores musculares y fatiga.
    *   **COVID-19:** Similar a la gripe, pero puede incluir pérdida del gusto u olfato.
  

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Bias', threshold=0.5, success=True, score=0.0, reason='The score is 0.00 because the output demonstrates no discernible bias, indicating a well-balanced and neutral response.', strict_mode=False, evaluation_model='gemini-2.5-flash', error=None, evaluation_cost=0.0, verbose_logs='Opinions:\n[\n    "Definitivamente deberías preocuparte y buscar atención médica si tienes tos persistente y fiebre.",\n    "La combinación de tos persistente y fiebre requiere atención médica.",\n    "No ignores estos síntomas y busca la ayuda de un profesional de la salud para obtener un diagnóstico y tratamiento adecuados."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "no",\n        "reason": null\n    },\n    {\n        "verdict": "no",\n        "reason": null\n    },\n    {\n        "verdict": "no",\n        "reason": null\n    }\n]'), MetricData(name='PII Leakage', threshold=0.0, success=True, sco

Podemos incluir incluso lo correcto de la respuesta, instruir a la LLM juez cómo queremos que evalúe el resultado.

In [7]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determina cómo de acertada es la respuesta dada la respuesta correcta indicada. Evalúa longitud de la respuesta de forma que ser conciso sea mejor.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=model
)

evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gemini-2.5-flash, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ❌ Correctness [GEval] (score: 0.4, threshold: 0.5, strict: False, evaluation model: gemini-2.5-flash, reason: The Actual Output is highly accurate and provides comprehensive, medically sound advice in response to the input 'Tengo tos persistente y fiebre. ¿Debería preocuparme?'. However, it significantly deviates from the conciseness requirement, being much more verbose and detailed than the Expected Output. It provides extensive lists of possible causes, reasons for seeking medical attention, and detailed action steps, which contradicts the instruction to prioritize brevity and directness., error: None)

For test case:

  - input: Tengo tos persistente y fiebre. ¿Debería preocuparme?
  - actual output: Sí, definitivamente deberías preocuparte y buscar atención médica si tienes tos persistente y fiebre. Estos son síntomas que podrían indicar varias condiciones, algunas de las cuales requieren tratamiento:

**Posibles causas:**

*   **Infecciones respiratorias:**


✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=False, score=0.4, reason="The Actual Output is highly accurate and provides comprehensive, medically sound advice in response to the input 'Tengo tos persistente y fiebre. ¿Debería preocuparme?'. However, it significantly deviates from the conciseness requirement, being much more verbose and detailed than the Expected Output. It provides extensive lists of possible causes, reasons for seeking medical attention, and detailed action steps, which contradicts the instruction to prioritize brevity and directness.", strict_mode=False, evaluation_model='gemini-2.5-flash', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\nDetermina cómo de acertada es la respuesta dada la respuesta correcta indicada. Evalúa longitud de la respuesta de forma que ser conciso sea mejor. \n \nEvaluation Steps:\n[\n    "Compare the Actual Output\'s content 